In [77]:
import numpy as np
import pandas as pd
import statsmodels as sm
import scipy  as sp
from statsmodels.formula.api import ols

In [2]:
def getFilePath(year):
    def paddingIfNeeded(num):
        if num < 10:
            return "0" + str(num)
        else:
            return str(num)
    path = "../ignore/MERGED" + str(year) + "_" + paddingIfNeeded((year + 1) % 100) +  "_PP.csv"
    return path
def getFile(path):
    tmp_file = pd.read_csv(path, sep=',', header=0, na_values=["NULL", "PrivacySuppressed", ""], keep_default_na=False, low_memory=False)
    tmp_file.rename(columns={'\ufeffUNITID': 'UNITID'}, inplace=True)
    return tmp_file

In [3]:
years = range(1996, 2015)
yearlyData = [getFile(getFilePath(year)) for year in years]
df = pd.concat(yearlyData, keys=years)

In [8]:
df.C200_L4.unique()

array([    nan,  0.1432,  0.1336, ...,  0.4233,  0.9727,  0.9624])

In [21]:
sp.stats.f_oneway(df[["C200_4"]], df[["ACCREDAGENCY"]])

ValueError: could not convert string to float: 'Council on Occupational Education'

In [50]:
list(df.columns.values)[1743:1784]

['ACCRED_AGENCY_Accreditation Commission for Education in Nursing Inc.',
 'ACCRED_AGENCY_Accrediting Bureau of Health Education Schools',
 'ACCRED_AGENCY_Accrediting Commission for Acupuncture and Oriental Medicine',
 'ACCRED_AGENCY_Accrediting Commission of Career Schools and Colleges',
 'ACCRED_AGENCY_Accrediting Council for Continuing Education & Training',
 'ACCRED_AGENCY_Accrediting Council for Independent Colleges and Schools',
 'ACCRED_AGENCY_American Bar Association',
 'ACCRED_AGENCY_American Board of Funeral Service Education Committee on Accreditation',
 'ACCRED_AGENCY_American Osteopathic Association',
 'ACCRED_AGENCY_American Podiatric Medical Association',
 'ACCRED_AGENCY_Association for Bibical Higher Educaiton',
 'ACCRED_AGENCY_Association of Advanced Rabbinical and Talmudic Schools',
 'ACCRED_AGENCY_Commission on Accrediting of the Association of Theological Schools',
 'ACCRED_AGENCY_Commission on Massage Therapy Accreditation',
 'ACCRED_AGENCY_Council on Accreditation 

In [58]:
cw_lm=ols('C200_L4 ~ C(CONTROL)', data=df).fit()
print(cw_lm.params)

Intercept            0.407559
C(CONTROL)[T.2.0]    0.279032
C(CONTROL)[T.3.0]    0.284498
dtype: float64


In [70]:
for i in range(3):
    tmp = str(5*(i+2))
    print(" ---- " + tmp)
    print(ols('C' + tmp + '0_4 ~ C(CONTROL)', data=df).fit().params)
    print(" ---- " + tmp + "L ")
    print(ols('C' + tmp + '0_L4 ~ C(CONTROL)', data=df).fit().params)

 ---- 10
Intercept            0.219905
C(CONTROL)[T.2.0]    0.160829
C(CONTROL)[T.3.0]   -0.004596
dtype: float64
 ---- 10L 
Intercept            0.266525
C(CONTROL)[T.2.0]    0.258777
C(CONTROL)[T.3.0]    0.138906
dtype: float64
 ---- 15
Intercept            0.435366
C(CONTROL)[T.2.0]    0.098473
C(CONTROL)[T.3.0]   -0.043735
dtype: float64
 ---- 15L 
Intercept            0.361301
C(CONTROL)[T.2.0]    0.267013
C(CONTROL)[T.3.0]    0.294254
dtype: float64
 ---- 20
Intercept            0.492120
C(CONTROL)[T.2.0]    0.056231
C(CONTROL)[T.3.0]   -0.154806
dtype: float64
 ---- 20L 
Intercept            0.407559
C(CONTROL)[T.2.0]    0.279032
C(CONTROL)[T.3.0]    0.284498
dtype: float64


In [71]:
for i in range(3):
    tmp = str(5*(i+2))
    print(" ---- " + tmp)
    print(ols('C' + tmp + '0_4 ~ C(ACCREDAGENCY)', data=df).fit().params)
    print(" ---- " + tmp + "L ")
    print(ols('C' + tmp + '0_L4 ~ C(ACCREDAGENCY)', data=df).fit().params)

 ---- 10
Intercept                                                                                                                                                                3.401787e-01
C(ACCREDAGENCY)[T.Accrediting Bureau of Health Education Schools]                                                                                                       -1.547677e-16
C(ACCREDAGENCY)[T.Accrediting Commission for Acupuncture and Oriental Medicine]                                                                                         -5.595694e-16
C(ACCREDAGENCY)[T.Accrediting Commission of Career Schools and Colleges]                                                                                                 9.884288e-02
C(ACCREDAGENCY)[T.Accrediting Council for Continuing Education & Training]                                                                                              -1.091660e-16
C(ACCREDAGENCY)[T.Accrediting Council for Independent Colleges and Schools]      

In [81]:
for i in range(3):
    tmp = str(5*(i+2))
    print(" ---- " + tmp)
    tmpLm = ols('C' + tmp + '0_4 ~ C(CONTROL):C(ACCREDAGENCY)', data=df).fit()
    print(sm.stats.anova.anova_lm(tmpLm, typ=2, transform=False))
    print(" ---- " + tmp + "L ")
    tmpLm = ols('C' + tmp + '0_L4 ~ C(CONTROL):C(ACCREDAGENCY)', data=df).fit()
    print(sm.stats.anova.anova_lm(tmpLm, typ=2, transform=False))

 ---- 10


AttributeError: 'DataFrame' object has no attribute 'design_info'

In [66]:
print(ols('C200_L4 ~ C(CONTROL)', data=df).fit().params)

Intercept            0.407559
C(CONTROL)[T.2.0]    0.279032
C(CONTROL)[T.3.0]    0.284498
dtype: float64


In [37]:
dummies = pd.get_dummies(df['ACCREDAGENCY']).rename(columns=lambda x: 'ACCRED_AGENCY_' + str(x))
df = pd.concat([df, dummies], axis=1)

In [51]:
dummies = pd.get_dummies(df['CONTROL']).rename(columns=lambda x: 'CONTROL_' + str(x))
df = pd.concat([df, dummies], axis=1)

In [56]:
list(df.columns.values)[1783:]

['CONTROL_1.0', 'CONTROL_2.0', 'CONTROL_3.0']

In [75]:
df.filter(regex='^CONTROL$|^ACCREDAGENCY$')

ACCREDAGENCY  CONTROL
1996 0                                                   NaN      1.0
     1                                                   NaN      1.0
     2                                                   NaN      1.0
     3                                                   NaN      1.0
     4                                                   NaN      2.0
     5                                                   NaN      1.0
     6                                                   NaN      1.0
     7                                                   NaN      1.0
     8                                                   NaN      1.0
     9                                                   NaN      1.0
     10                                                  NaN      1.0
     11                                                  NaN      1.0
     12                                                  NaN      1.0
     13                                                  NaN      2.0
     14                                                  NaN      1.0
     15                                                  NaN      3.0
     16                                                  NaN      1.0
     17                                                  NaN      1.0
     18                                                  NaN      2.0
     19                                                  NaN      1.0
     20                                                  NaN      3.0
     21                                                  NaN      1.0
     22                                                  NaN      1.0
     23                                                  NaN      2.0
     24                                                  NaN      2.0
     25                                                  NaN      2.0
     26                                                  NaN      2.0
     27                                                  NaN      3.0
     28                                                  NaN      1.0
     29                                                  NaN      3.0
...                                                      ...      ...
2014 7673  National Accrediting Commission of Cosmetology...      3.0
     7674                  Council on Occupational Education      3.0
     7675       Middle States Commission on Higher Education      3.0
     7676       Middle States Commission on Higher Education      3.0
     7677       Middle States Commission on Higher Education      3.0
     7678       Middle States Commission on Higher Education      3.0
     7679       Middle States Commission on Higher Education      3.0
     7680       Middle States Commission on Higher Education      3.0
     7681       Middle States Commission on Higher Education      3.0
     7682       Middle States Commission on Higher Education      3.0
     7683       Middle States Commission on Higher Education      3.0
     7684       Middle States Commission on Higher Education      3.0
     7685       Middle States Commission on Higher Education      3.0
     7686       Middle States Commission on Higher Education      3.0
     7687       Middle States Commission on Higher Education      3.0
     7688       Middle States Commission on Higher Education      3.0
     7689       Middle States Commission on Higher Education      3.0
     7690       Middle States Commission on Higher Education      3.0
     7691       Middle States Commission on Higher Education      3.0
     7692       Middle States Commission on Higher Education      3.0
     7693  Accrediting Commission of Career Schools and C...      3.0
     7694     Accrediting Bureau of Health Education Schools      2.0
     7695                  Council on Occupational Education      3.0
     7696                  Council on Occupational Education      3.0
     7697  Accrediting Council for Independent Colleges a...      3.0
     7698  North Central Association of Colleges and Scho.

In [83]:
df.CCBASIC.unique()

array([ nan,  18.,  15.,  20.,  16.,  19.,   1.,  22.,  21.,   8.,  -2.,
         7.,   9.,  23.,  24.,   4.,   5.,  10.,  32.,  13.,  30.,  29.,
        26.,   6.,   2.,  17.,  28.,  33.,  11.,   3.,  25.,  12.,  14.,
        31.,  27.])